  ### FINAL ASSIGNMENT                                                     
  ##### Rizwan Saleem

#### Description of Assignment

1)  Create RDF from the GFF and store it in BlazeGraph.  Use this BlazeGraph database to provide the data for your website.

2)  Pull-in information from ENSEMBL about the Magnaporthe gene associated with each PolyA site and present this information to the biologist on your web page.  You may choose to either import this data at the beginning, create RDF from it, and add it to your database, OR you can pull-in the gene information dynamically through your Web script.  It's entirely your choice, and I will be happy with either solution.  You can either use the BioRuby Seq Feature object, or you can use your own Gene object, whichever you prefer (your choice of object will likely be determined by your choice to bring the data in at the beginning, or bring it in dynamically... using BioRuby is the obvious choice if you are reading EMBL files, but your own Gene object is probably a better choice if you are getting the gene information from your own SPARQL query)

3)  You must think of a way to show Dr. Sesma the differential expression of each PolyA site under each conditions (a polyA site may be found in all 4 conditions, or it may exist in only one or two growth conditions... you need to show her which sites exist in which conditions)

4)  Define each polyA site as being Intronic, Exonic, 5'UTR, 3'UTR, or "other"

Things you will need to consider:

You need to create URL's for each PolyA site when you design your RDF - use http://localhost/******** as the base of your URLs.  Think carefully about how this URL looks!  (e.g. think about RESTful URLs, and think about what happens when someone clicks on that URL)
Design RDF and select your predicates and rdf:type ontology classes.  One useful tool to find useful ontology classes is the EBI Ontology Lookup Service (http://www.ebi.ac.uk/ontology-lookup/).  For predicates, you will probably have to invent these... (and it is 100% OK with me if you invent them!)
You will need some starting point for your biologist... e.g. create a starting Web page where your biologist can, for example, enter the ID of the gene they are interested in.  When they click "submit" they will be taken to a page about that gene, including the list of PolyA sites in that gene, and the differential expression of those polyA sites.

Following required:

In [1]:
require './gene.rb' # Gene object (.rb file apart)
require 'net/http'
require 'rest-client'
require 'bio' # to call Bioruby

true

The first set is to creat the RDF from GFF file for this purpose first read the GFF file. Then we have to count the different gene uploaded from GFFs.

In [2]:
# to read the GFF files
Gene.load_gene_ids("WT--C-X_polyA.gff", "WT--C")
Gene.load_gene_ids("WT--N-X_polyA.gff", "WT--N")
Gene.load_gene_ids("WT-CM-X_polyA.gff", "WT-CM")
Gene.load_gene_ids("WT-MM-X_polyA.gff", "WT-MM")

[#<Gene:0x00000000028a09b0 @gene_id="MGG_17623", @chromosome="6", @location="1000217", @strand="-", @conditions=["WT--C"]>, #<Gene:0x00000000028a0190 @gene_id="MGG_03694", @chromosome="4", @location="1001280", @strand="+", @conditions=["WT--C", "WT--N", "WT-CM", "WT-MM"]>, #<Gene:0x00000000028b38d0 @gene_id="MGG_00958", @chromosome="5", @location="1002880", @strand="+", @conditions=["WT--C", "WT--N", "WT-CM", "WT-MM"]>, #<Gene:0x00000000028b3330 @gene_id="MGG_00958", @chromosome="5", @location="1002926", @strand="+", @conditions=["WT--C", "WT--N", "WT-CM", "WT-MM"]>, #<Gene:0x00000000028b2a98 @gene_id="MGG_13328", @chromosome="3", @location="100456", @strand="+", @conditions=["WT--C", "WT--N", "WT-MM"]>, #<Gene:0x00000000028b22c8 @gene_id="MGG_02222", @chromosome="1", @location="1005066", @strand="-", @conditions=["WT--C"]>, #<Gene:0x00000000028b1b98 @gene_id="MGG_02926", @chromosome="7", @location="1005376", @strand="-", @conditions=["WT--C", "WT--N", "WT-CM"]>, #<Gene:0x00000000028b1

In [ ]:
Gene.how_many # to count the total number of different genes uploaded from the GFFs

In [ ]:
genes = Gene.genes

In [ ]:
# to check that our 'Gene' object has been correctly created (just a few examples!)
puts genes[0].gene_id + "\t" + genes[0].chromosome + "\t" + genes[0].location + "\t" + genes[0].strand + "\t" + genes[0].conditions.join(', ')
puts genes[1].gene_id + "\t" + genes[1].chromosome + "\t" + genes[1].location + "\t" + genes[1].strand + "\t" + genes[1].conditions.join(', ')
puts genes[2].gene_id + "\t" + genes[2].chromosome + "\t" + genes[2].location + "\t" + genes[2].strand + "\t" + genes[2].conditions.join(', ')
puts genes[3].gene_id + "\t" + genes[3].chromosome + "\t" + genes[3].location + "\t" + genes[3].strand + "\t" + genes[3].conditions.join(', ')
puts genes[4].gene_id + "\t" + genes[4].chromosome + "\t" + genes[4].location + "\t" + genes[4].strand + "\t" + genes[4].conditions.join(', ')

In [ ]:
embl_records = Array.new() # to create an array for storing the Bio::EMBL records
genes.each do |gene|
  # to call dbfetch for each instance of the 'Gene' object
  address = URI("http://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=ensemblgenomesgene&id=#{gene.gene_id}&format=embl")  # create a "URI" object (Uniform Resource Identifier: https://en.wikipedia.org/wiki/Uniform_Resource_Identifier)
  response = Net::HTTP.get_response(address) # to use the Net::HTTP object "get_response" method
  if response
    record = response.body
    entry = Bio::EMBL.new(record) # to create a Bio::EMBL' objects
    embl_records << [gene, entry] # to record each gene + entry in the embl_records array
  end
end

In [ ]:
# to check that everything is OK so far...
embl_records.each do |record|
  puts record[0]
  puts record[1]
end

In [ ]:
annotated_records = Array.new()
embl_records.each do |record| # to loop over each Bio::EMBL object
  f = Bio::Feature.new('poly A', record[0].location) # to generate  new feature ('polyA' motif) for each record
                                                     # using the fields of the GFF file
  f.append(Bio::Feature::Qualifier.new('associated_gene', record[0].gene_id))
  f.append(Bio::Feature::Qualifier.new('chromosome', record[0].chromosome))
  f.append(Bio::Feature::Qualifier.new('strand', record[0].strand))
  f.append(Bio::Feature::Qualifier.new('expression_conditions', record[0].conditions))
  record[1].features << f
  annotated_records << record[1]
end

In [ ]:
annotated_records.each do |record| # to loop over each Bio::EMBL object
  record.features.each do |feature| # to loop over each Bio::Feature
    if feature.is_a?(Bio::Feature) && feature.feature == "poly A" # to specifically take 'exon' features
      feature.each do |qualifier|
        if qualifier.qualifier != "expression_conditions" 
          puts qualifier.qualifier + ": " + qualifier.value # to get information about chromosomes, strand, etc.
        elsif qualifier.qualifier == "expression_conditions"
          puts qualifier.qualifier + ": " + qualifier.value.join(', ') # to get the expression conditions of each polyA motif    
        end
      end
      puts feature.position # to get the position of the feature
      puts "*****"
    end    
  end
end

In [ ]:
# to retrieve the description of each record in the Bio::EMBL object
puts annotated_records[0].description
puts annotated_records[1].description
puts annotated_records[2].description
puts annotated_records[3].description
puts annotated_records[4].description

In [ ]:
## NEXT STEP: use the information obtained so far to create the RDF triples required to run your website!!!